# PDGA Results

## *<u>Documentation</u>*

[Authorization](https://www.pdga.com/dev/api/rest/v1/auth)

[API call examples](https://www.pdga.com/dev/api/rest/v1/services)

> **Note**: The PDGA Developer Program is currently closed. We are not accepting new applications for the PDGA Developer Program at this time. We do anticipate opening up the program again in the future once development of an updated version of the API is complete.


## *<u>Import libraries</u>*

In [90]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools

In [ ]:
def tag_dict(data: list):
    return {f'tag_{i}': {tag} for i,tag in enumerate(data)}

In [141]:
def default_tag_list():
    return [
            {
                'name': 'Table',
                'tags': [
                    {'object': 'table', 'element': 'id', 'tag': 'tournament-stats'}
                ], 
                'in_results': False
            },
            {
                'name': 'Row',
                'tags': [
                    {'object': 'tr', 'element': 'class', 'tag': 'odd'},
                    {'object': 'tr', 'element': 'class', 'tag': 'even'}
                ], 
                'in_results': False
            },
            {
                'name': 'Score',
                'tags': [
                    {'object': 'td', 'element': 'class', 'tag': 'par'},
                    {'object': 'td', 'element': 'class', 'tag': 'even'},
                    {'object': 'td', 'element': 'class', 'tag': 'dnf'}
                ], 
                'in_results': True
            },
            {
                'name': 'Place',
                'tags': [
                    {'object': 'td', 'element': 'class', 'tag': 'place'}
                ], 
                'in_results': True
            },
            {
                'name': 'Player',
                'tags': [
                    {'object': 'td', 'element': 'class', 'tag': 'player'}
                ], 
                'in_results': True
            },
            {
                'name': 'PDGA Number',
                'tags': [
                    {'object': 'td', 'element': 'class', 'tag': 'pdga-number'}
                ], 
                'in_results': True
            },
            {
                'name': 'Player Rating',
                'tags': [
                    {'object': 'td', 'element': 'class', 'tag': 'player-rating'}
                ], 
                'in_results': True
            },
            # {'Name': 'Player Rating', 'Tags': ['player-rating propagator']}
        ]

In [153]:
def soupify(url):
    page = requests.get(url)
    soup = bs(page.content, "html.parser")
    return soup

In [163]:
def search_string(_object, _element, _tag):
    return f'{_object}[{_element}*="{_tag}"]'

In [199]:
def html_parser(soup, statement, one=True):
    parsed = soup.select(statement)
    
    if one:
        return parsed[0]
    else:
        return parsed

In [200]:
test.tags

[{'name': 'Table',
  'tags': [{'object': 'table', 'element': 'id', 'tag': 'tournament-stats'}],
  'in_results': False},
 {'name': 'Row',
  'tags': [{'object': 'tr', 'element': 'class', 'tag': 'odd'},
   {'object': 'tr', 'element': 'class', 'tag': 'even'}],
  'in_results': False},
 {'name': 'Score',
  'tags': [{'object': 'td', 'element': 'class', 'tag': 'par'},
   {'object': 'td', 'element': 'class', 'tag': 'even'},
   {'object': 'td', 'element': 'class', 'tag': 'dnf'}],
  'in_results': True},
 {'name': 'Place',
  'tags': [{'object': 'td', 'element': 'class', 'tag': 'place'}],
  'in_results': True},
 {'name': 'Player',
  'tags': [{'object': 'td', 'element': 'class', 'tag': 'player'}],
  'in_results': True},
 {'name': 'PDGA Number',
  'tags': [{'object': 'td', 'element': 'class', 'tag': 'pdga-number'}],
  'in_results': True},
 {'name': 'Player Rating',
  'tags': [{'object': 'td', 'element': 'class', 'tag': 'player-rating'}],
  'in_results': True}]

In [207]:
class Scraper:
    
    def __init__(
        self,
        event_number: int = 1,
        base_url: str = 'https://www.pdga.com/tour/event/',
        url_addon: str = '',
        tags: list = default_tag_list()
    ):
        
        self.event_number = event_number
        self.base_url = base_url
        self.url_addon = url_addon
        self.event_url = self.base_url + self.url_addon + str(self.event_number)
        self.tags = tags
        self.results_columns = []
        self.raw_data = ''
        
        def tags_parser(self, name, tags, in_results):
            setattr(self, name, tags)
                
            if in_results:
                self.results_columns.append(name)
            # print(f'{name} tags have been set')
        
        for tag in self.tags:
            tags_parser(self, tag['name'].replace(' ',''), tag['tags'], tag['in_results'])
            
            
    def results(self):
        soup = soupify(self.event_url)
#         print(type(soup.prettify()))
#         print(soup.prettify())
        
        for tag in self.Table:
            search = search_string(tag['object'], tag['element'], tag['tag'])
            self.raw_data += str(html_parser(soup, search, one=True))
            
        
        print(self.raw_data.prettify())
        
#         results_table_raw = soup.select(f"{self.Table['object']}[self.*="tournament-stats"]')
#         odd_rows = results_table_raw.find_all(class_="odd")
#         even_rows = results_table_raw.find_all(class_="even")
#         results_raw = [x for x in itertools.chain.from_iterable(itertools.zip_longest(odd_rows,even_rows)) if x]
#         results_list = [parse_rows(row) for row in results_raw]
#         results_df = pd.DataFrame(data=results_list).set_index('Place')
#         return results_df
            
        
    def parser(data):
        if data.select(f'td[class*="{self.score_tags.tag_1}"]'):
            score = data.select(f'td[class*="{self.score_tags.tag_1}"]')[0].text
        elif data.select(f'td[class*="{self.score_tags.tag_2}"]'):
            score = self.score_tags.tag_2.upper()
        else:
            score = "ERROR"
            
        result = { # could also do round scores, round ratings, and total score
            'Place': int(data.find(class_=self.place_tag).text),
            'Player': data.find(class_=self.player_tag).text,
            'Score': score
        }
        
        for tag in self.additional_tags:
            result[tag.Header] = tag.Tag
            
        return result

In [208]:
test = Scraper('64957')

In [209]:
test.results()

AttributeError: 'str' object has no attribute 'prettify'

In [210]:
def parse_rows(row):
    if row.select('td[class*="par"]'):
        score = row.select('td[class*="par"]')[0].text
    elif row.select('td[class*="dnf"]'):
        score = "DNF"
    else:
        score = "ERROR"
    result = { # could also do round scores, round ratings, and total score
        'Place': int(row.find(class_="place").text),
        'Player': row.find(class_="player").text,
        'PDGA Number': int(row.find(class_="pdga-number").text),
        'Player Rating': int(row.find(class_="player-rating propagator").text),
        'Score': score,
    }
    return result

In [211]:
def results(event_number):
    base_url = 'https://www.pdga.com/tour/event/'
    url = base_url + str(event_number)
    page = requests.get(url)
    soup = bs(page.content, "html.parser")
    results_table_raw = soup.find_all(class_="table-container")[1]
    odd_rows = results_table_raw.find_all(class_="odd")
    even_rows = results_table_raw.find_all(class_="even")
    results_raw = [x for x in itertools.chain.from_iterable(itertools.zip_longest(odd_rows,even_rows)) if x]
    results_list = [parse_rows(row) for row in results_raw]
    results_df = pd.DataFrame(data=results_list).set_index('Place')
    return results_df

In [212]:
event_number = 64957

In [213]:
results_df = results(event_number)
results_df

,Player,PDGA Number,Player Rating,Score
Place,,,,
1,Kyle Klein,85132,1039,-27
2,Bradley Williams,31644,1033,-26
3,Isaac Robinson,50670,1041,-23
3,Simon Lizotte,8332,1042,-23
5,Calvin Heimburg,45971,1051,-22
...,...,...,...,...
105,Casey Hanemayer,67314,1017,+27
106,Dennis Augustsson,98130,1015,+28
107,Gavin Rathbun,60436,1021,+29


In [95]:
results_df.to_csv('Results_USDGC.csv')